# Transfer Learning

In [22]:
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline
from keras.datasets import fashion_mnist

In [23]:
def fashion_mnist_preprocess():
    
    num_classes = 10
    
    # get data
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train.shape, y_train.shape, x_test.shape, y_test.shape
    
    # input image dimensions
    img_rows, img_cols = 28, 28
    
    # reshape to channel_last format
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    # normalize data
    x_train_normalize = x_train / 255
    x_test_normalize = x_test / 255
    
    # convert class vectors to binary class matrices
    y_train_class = keras.utils.to_categorical(y_train, num_classes)
    y_test_class = keras.utils.to_categorical(y_test, num_classes)
    
    return (x_train_normalize, y_train_class), (x_test_normalize, y_test_class)

In [24]:
(x_train_conv, y_train_class), (x_test_conv, y_test_class) = fashion_mnist_preprocess()

## Basic CNN

In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [26]:
cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(10, activation='softmax'))

In [27]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 3, 128)         0         
__________

In [28]:
cnn.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [29]:
%time
#history = cnn.fit(x_train_conv, y_train_class, batch_size=128, epochs=20, verbose=1, validation_split = 0.2)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.6 µs


In [30]:
def plot_metric(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.ylim(0,1)
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.ylim(0,1)
    plt.legend()

    plt.show()

In [31]:
score = cnn.evaluate(x_test_conv, y_test_class, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.3050410022735597
Test accuracy: 0.0986


## Data Augementation

In [32]:
from keras.preprocessing.image import ImageDataGenerator

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_conv, y_train_class, test_size=0.2, random_state=12345)

In [34]:
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
datagen.fit(x_train)

In [1]:
# fits the model on batches with real-time data augmentation:
#history = cnn.fit_generator(datagen.flow(x_train_conv, y_train_class, batch_size=32), 
                            validation_data=(x_val, y_val), use_multiprocessing=True,
                    steps_per_epoch=len(x_train_conv) / 32, epochs=2)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [2]:
plot_metric(history)

NameError: name 'plot_metric' is not defined

In [3]:
score = cnn.evaluate(x_test_conv, y_test_class, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'cnn' is not defined

## Transfer Learning

In [5]:
from keras.applications import inception_v3

In [8]:
conv_base = inception_v3.InceptionV3(include_top=False, weights='imagenet', 
                        input_tensor=None, input_shape=None, pooling=None, classes=1000)

87916544/87910968 [==============================] - 1s 0us/step


In [9]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 